<a href="https://colab.research.google.com/github/alerodriguessf/brazil-presidential-agenda-analysis/blob/main/what_happens_in_the_dark_20250425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ydata-profiling

In [ ]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
df = pd.read_excel('agenda-do-presidente-da-republica-2021 - agenda-do-presidente-da-republica-2021.csv.xlsx')

In [ ]:
df.head(30)

In [ ]:
df.info()

In [ ]:
ProfileReport(df)

In [ ]:
df_clean = df.drop(columns=['Descrição da Agenda','Nome do Órgão','Nome da Autoridade','Tipo de agenda','Título da Agenda'])

In [ ]:
df_clean.head()

In [ ]:
df_clean.info()

In [ ]:
df_clean.dropna(subset=['Horário de Inicio','Horário de Término','Duração','Data da Agenda Diária'])

In [ ]:
ProfileReport(df_clean)

In [ ]:
df_clean.info()

In [ ]:
df_clean['Horário de Inicio'] = pd.to_datetime(df_clean['Horário de Inicio'], format = '%H:%M:%S').dt.time
df_clean['Horário de Término'] = pd.to_datetime(df_clean['Horário de Término'], format = '%H:%M:%S').dt.time

In [ ]:
def converter_duracao(duracao_str):
    # Check if the input is a string and not NaN
    if isinstance(duracao_str, str):
        try:
            # Attempt to split and convert to integers
            horas, minutos = map(int, duracao_str.split(':'))
            total_segundos = (horas * 3600) + (minutos * 60)
            return pd.to_timedelta(total_segundos, unit='s')
        except ValueError:
            # Handle cases where the string format is incorrect (e.g., not HH:MM)
            print(f"Could not parse duration string: {duracao_str}")
            return pd.NaT # Return Not a Time for parsing errors
    else:
        # Handle non-string inputs (like NaN after conversion)
        # print(f"Non-string input received: {duracao_str}") # Optional: for debugging
        return pd.NaT # Return Not a Time for non-string values

# Convert the 'Duração' column to string type first.
# This handles potential integers, floats, or NaNs by turning them into their string representation.
# Then, apply the converter function.
df_clean['Duração'] = df_clean['Duração'].astype(str).apply(converter_duracao)

In [ ]:
df_clean.info()

In [ ]:
print(df_clean[['Horário de Inicio', 'Horário de Término', 'Duração']].dtypes)


In [ ]:
df_clean.head()

Entendo o Perfil dos lugares/viajens internacionais

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Pré-processamento (assumindo que df_clean já está pronto)
# Contar a frequência de cada local
local_counts = df_clean['Local do Compromisso'].value_counts()

# Selecionar os top N locais (para evitar um gráfico muito longo)
top_n = 20  # Ajuste este valor se quiser mostrar mais ou menos locais
top_local_counts = local_counts.head(top_n)

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))  # Tamanho da figura ajustável
sns.barplot(x=top_local_counts.values, y=top_local_counts.index, palette='viridis')  # Cores agradáveis

# Adicionar rótulos e título
plt.xlabel('Número de Compromissos', fontsize=12)
plt.ylabel('Local do Compromisso', fontsize=12)
plt.title(f'Top {top_n} Locais de Compromisso', fontsize=14)

# Estilizar o gráfico
sns.despine(left=True, bottom=True)  # Remover spines desnecessárias
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()  # Ajustar layout para evitar cortes

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Pré-processamento (assumindo que df_clean já está pronto)
df_analise = df_clean.copy()

# Ensure the 'Data da Agenda Diária' column is in datetime format
# Use errors='coerce' to turn invalid parsing into NaT (Not a Time)
df_analise['Data da Agenda Diária'] = pd.to_datetime(df_analise['Data da Agenda Diária'], format='%d/%m/%Y', errors='coerce')

# Ensure the time columns are in datetime.time format
# You can keep this as it is if you already did it before, but ensure it happened
df_analise['Horário de Inicio'] = pd.to_datetime(df_analise['Horário de Inicio'], errors='coerce', format='%H:%M:%S').dt.time
df_analise['Horário de Término'] = pd.to_datetime(df_analise['Horário de Término'], errors='coerce', format='%H:%M:%S').dt.time


# Calcular a duração de cada compromisso em horas
# Combine the date with the time objects for calculation
df_analise['Duração (horas)'] = df_analise.apply(
    lambda row: (
        (pd.to_datetime(str(row['Data da Agenda Diária']).split()[0] + ' ' + str(row['Horário de Término'])) -
         pd.to_datetime(str(row['Data da Agenda Diária']).split()[0] + ' ' + str(row['Horário de Inicio']))).total_seconds() / 3600
        if pd.notna(row['Horário de Inicio']) and pd.notna(row['Horário de Término']) and pd.notna(row['Data da Agenda Diária'])
        else 0  # Se algum horário ou data for nulo, considerar duração 0
    ),
    axis=1,
)

# Calcular o total de horas no mandato (considerando apenas os dias em df_clean)
# Ensure the date column is handled correctly for min/max
inicio_mandato = df_analise['Data da Agenda Diária'].min()
fim_mandato = df_analise['Data da Agenda Diária'].max()
# Calculate total hours between the first and last date
total_horas_mandato = (fim_mandato - inicio_mandato).total_seconds() / 3600 if pd.notna(inicio_mandato) and pd.notna(fim_mandato) else 0


# Agrupar por local e somar as durações
horas_por_local = df_analise.groupby('Local do Compromisso')['Duração (horas)'].sum().reset_index()

# Calcular a proporção do total de horas do mandato
horas_por_local['Proporção'] = horas_por_local['Duração (horas)'] / total_horas_mandato if total_horas_mandato > 0 else 0

# Selecionar os top N locais para visualização
top_n = 10
top_locais = horas_por_local.nlargest(top_n, 'Duração (horas)')

# Visualizar (Gráfico de Barras)
plt.figure(figsize=(10, 6))
sns.barplot(x='Proporção', y='Local do Compromisso', data=top_locais, palette='viridis')

plt.xlabel('Proporção do Tempo do Mandato', fontsize=12)
plt.ylabel('Local do Compromisso', fontsize=12)
plt.title(f'Proporção do Tempo do Mandato nos Top {top_n} Locais', fontsize=14)
sns.despine(left=True, bottom=True)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

# Mostrar os resultados em texto também
print("\nProporção do tempo do mandato por local:")
print(horas_por_local.sort_values(by='Proporção', ascending=False).head(10))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Pré-processamento (assumindo que df_clean e o código anterior já foram executados)
# (Ou seja, você já tem o DataFrame 'horas_por_local' com as proporções)

# Selecionar os top N locais para visualização (novamente, para clareza)
top_n = 10
top_locais = horas_por_local.nlargest(top_n, 'Duração (horas)')

# Criar o gráfico de rosca
plt.figure(figsize=(8, 8))
plt.pie(
    top_locais['Proporção'],
    labels=top_locais['Local do Compromisso'],
    autopct='%1.1f%%',  # Formatar porcentagens
    startangle=140,    # Ajustar o ângulo inicial
    colors=sns.color_palette('pastel', n_colors=top_n), # Paleta de cores
    wedgeprops={'width': 0.3} # Largura da rosca
)

# Adicionar título
plt.title(f'Porcentagem do Tempo do Mandato nos Top {top_n} Locais', fontsize=14)

# Ajustar layout
plt.tight_layout()
plt.show()

**VIZUALIZAÇÕES (EXPERIMENTAL)**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Pré-processamento (assumindo que df_clean já está pronto)
df_plot = df_clean.copy()
df_plot['Data'] = pd.to_datetime(df_plot['Data da Agenda Diária'], format='%d/%m/%Y')
df_plot['Hora Inicio'] = df_plot['Horário de Inicio'].apply(lambda t: t.hour + t.minute / 60 if pd.notna(t) else None)
df_plot['Hora Termino'] = df_plot['Horário de Término'].apply(lambda t: t.hour + t.minute / 60 if pd.notna(t) else None)
df_plot['Dia da Semana'] = df_plot['Data'].dt.dayofweek  # 0: Segunda (0) a Domingo (6)

# Filtrar apenas dias úteis (Segunda a Sexta)
df_plot = df_plot[df_plot['Dia da Semana'].isin(range(0, 5))]

# Criar intervalos de tempo (horas comerciais: 6h - 24h)
horas = list(range(6, 25))  # Inclui a hora 24
df_heatmap = pd.DataFrame({'Hora': horas})
df_heatmap['Data'] = df_heatmap['Hora'].apply(lambda x: df_plot['Data'].unique())
df_heatmap = df_heatmap.explode('Data')
df_heatmap['Ocupado'] = 0

for index, row in df_plot.iterrows():
    if pd.notna(row['Hora Inicio']) and pd.notna(row['Hora Termino']):
        start_hour = int(row['Hora Inicio'])
        end_hour = int(row['Hora Termino']) + 1  # Incluir a hora final
        # Filtrar para horas comerciais
        start_hour = max(start_hour, 6)
        end_hour = min(end_hour, 24)
        if start_hour < end_hour:
            df_heatmap.loc[(df_heatmap['Data'] == row['Data']) & (df_heatmap['Hora'].isin(range(start_hour, end_hour))), 'Ocupado'] = 1

df_heatmap_pivot = df_heatmap.pivot_table(index='Data', columns='Hora', values='Ocupado', fill_value=0)

# Visualização com Tabela 5x Mais Larga
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(100,100))  # Largura 5x maior (10 * 5 = 50)

sns.heatmap(
    df_heatmap_pivot,
    cmap=['#333333', 'deepskyblue'],
    cbar=False,
    linewidths=0.5,
    linecolor='#555555',
    square=True,
    ax=ax,
)

# Ajustar os eixos (Horas Comerciais e Datas)
ax.set_xticks(range(6, 25))
ax.set_xticklabels(horas, fontsize=8, color='lightgray')
ax.set_xlabel('Hora (6h - 24h)', fontsize=10, color='white')

# Ajustar rótulos do eixo y (Datas) - Mais compacto
num_dias = len(df_heatmap_pivot.index)
step_size = max(1, num_dias // 20)
ax.set_yticks(range(0, num_dias, step_size))
ax.set_yticklabels(df_heatmap_pivot.index[::step_size].strftime('%Y-%m-%d'), fontsize=8, color='lightgray')
ax.set_ylabel('Data', fontsize=10, color='white')

# Título e Legenda
plt.title('Agenda (Dias Úteis, 6h-24h)', fontsize=12, color='white', pad=10)
plt.figtext(0.85, 0.15, 'Legenda:', fontsize=10, color='white', ha='left')
plt.figtext(0.85, 0.13, 'Livre', fontsize=10, color='lightgray', ha='left')
plt.figtext(0.85, 0.11, 'Ocupado', fontsize=10, color='deepskyblue', ha='left')

plt.tight_layout(rect=[0, 0, 0.83, 0.95])  # Ajustar layout para a legenda
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# Pré-processamento (assumindo que df_clean já está pronto)
df_plot = df_clean.copy()
df_plot['Data'] = pd.to_datetime(df_plot['Data da Agenda Diária'], format='%d/%m/%Y')
df_plot['Hora Inicio'] = df_plot['Horário de Inicio'].apply(lambda t: t.hour + t.minute / 60 if pd.notna(t) else None)
df_plot['Hora Termino'] = df_plot['Horário de Término'].apply(lambda t: t.hour + t.minute / 60 if pd.notna(t) else None)
df_plot['Dia da Semana'] = df_plot['Data'].dt.dayofweek  # 0: Segunda (0) a Domingo (6)

# Filtrar apenas dias úteis (Segunda a Sexta)
df_plot = df_plot[df_plot['Dia da Semana'].isin(range(0, 5))]

# Criar intervalos de tempo (horas comerciais: 6h - 24h)
horas = list(range(6, 25))  # Inclui a hora 24
df_heatmap = pd.DataFrame({'Hora': horas})
df_heatmap['Data'] = df_heatmap['Hora'].apply(lambda x: df_plot['Data'].unique())
df_heatmap = df_heatmap.explode('Data')
df_heatmap['Ocupado'] = 0

for index, row in df_plot.iterrows():
    if pd.notna(row['Hora Inicio']) and pd.notna(row['Hora Termino']):
        start_hour = int(row['Hora Inicio'])
        end_hour = int(row['Hora Termino']) + 1  # Incluir a hora final
        # Filtrar para horas comerciais
        start_hour = max(start_hour, 6)
        end_hour = min(end_hour, 24)
        if start_hour < end_hour:
            df_heatmap.loc[(df_heatmap['Data'] == row['Data']) & (df_heatmap['Hora'].isin(range(start_hour, end_hour))), 'Ocupado'] = 1

df_heatmap_pivot = df_heatmap.pivot_table(index='Data', columns='Hora', values='Ocupado', fill_value=0)

# Visualização Maximizada para Celular
plt.style.use('dark_background')

# Calcular tamanho ideal da figura e dos quadrados
num_dias = df_heatmap_pivot.shape[0]
num_horas = df_heatmap_pivot.shape[1]
square_size = 30  # Tamanho base do quadrado (ajuste conforme necessário)
fig_width = num_horas * square_size / 72  # Converter pixels para polegadas (72 DPI)
fig_height = num_dias * square_size / 72

fig, ax = plt.subplots(figsize=(fig_width, fig_height))

sns.heatmap(
    df_heatmap_pivot,
    cmap=['#333333', 'deepskyblue'],
    cbar=False,
    linewidths=0,  # Sem linhas entre os quadrados
    linecolor=None,
    square=True,
    ax=ax,
)

# Ajustar os eixos (Horas e Datas)
ax.set_xticks(np.arange(0.5, num_horas + 0.5, 1))  # Centralizar rótulos
ax.set_xticklabels(horas, fontsize=1, color='lightgray')  # Rótulos de hora muito pequenos
ax.set_xlabel('Hora (6h - 24h)', fontsize=6, color='white')

ax.set_yticks(np.arange(0.5, num_dias + 0.5, 5))  # Centralizar rótulos
ax.set_yticklabels(df_heatmap_pivot.index[::5].strftime('%Y-%m-%d'), fontsize=1, color='lightgray')  # Rótulos de data muito pequenos
ax.set_ylabel('Data', fontsize=6, color='white')

# Remover ticks e spines
ax.tick_params(length=0, pad=0)
for spine in ax.spines:
    ax.spines[spine].set_visible(False)

# Legenda Eficiente
plt.figtext(0.90, 0.90, 'Legenda:', fontsize=8, color='white', ha='left')
plt.figtext(0.90, 0.88, 'Livre', fontsize=8, color='lightgray', ha='left')
plt.figtext(0.90, 0.86, 'Ocupado', fontsize=8, color='deepskyblue', ha='left')

plt.title('Agenda (Dias Úteis, 6h-24h)', fontsize=10, color='white', pad=5)
plt.tight_layout(rect=[0, 0, 0.88, 0.95], pad=0)  # Remover padding externo
plt.subplots_adjust(wspace=0, hspace=0)  # Remover padding interno

plt.show()